In [18]:
!pip install dask

In [10]:
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
import planetary_computer as pc
import sys
sys.path.append('/home/jovyan/data-ingestion') 
import utils.sentinel as sentinel
import pandas as pd
import geopandas as gpd
import numpy as np
import fastparquet

import dask.dataframe as dd
from adlfs import AzureBlobFileSystem
# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)

In [11]:
# Not used directly, but either fastparquet or pyarrow needs to be installed
# import fastparquet

storage_account_name = 'cpdataeuwest'
folder_name = 'cpdata/raw/fia'

fs = AzureBlobFileSystem(account_name=storage_account_name)
parquet_files = fs.glob(folder_name + '/*parquet')


In [16]:
filename = 'plot'
df = dd.read_parquet(f"az://{folder_name}/{filename}.parquet",\
                     engine='fastparquet',\
                     storage_options={'account_name':storage_account_name}).compute()
df.head()
#filter here for data only in sentinel range
df = df[df['MEASYEAR']>2014]
df['DATETIME'] = [pd.to_datetime(f"{int(i['MEASYEAR'])}-{int(i['MEASMON'])}-{int(i['MEASDAY'])}", format='%Y-%m-%d') for _,i in df.iterrows()]

ValueError: Unknown protocol az

In [4]:
# import pandas as pd
# import geopandas as gpd
# import numpy as np



In [153]:
#create the time-of-interest
df['DATERANGE'] = create_daterange(df)

selections = ['CN', 'INVYR', 'LAT', 'LON', 'DATETIME', 'DATERANGE']

subdf = df[selections].dropna()
gdf = gpd.GeoDataFrame(
    subdf, geometry=gpd.points_from_xy(subdf.LON, subdf.LAT))

gdf = gdf.set_crs('EPSG:4326')

buffer_distance = 6400 #buffer distance of 6400m to attain a 640x640 chip at 10m resolution
#we need to make a big enough buffer to encompass the 5km range
#ideally, we will make a chip at the largest 1400m buffer, so the largest chip will be 140x140, or 128x128
gdf = apply_buffer_to_points(gdf, buffer_distance)
#create the area-of-interest
gdf['AOI'] = [create_aoi(i['geometry']) for _,i in gdf.iterrows()]

#check the area size of the aoi
#gdf_m = gdf.to_crs("EPSG:3857")
#np.sqrt(gdf_m.area/1e06)

In [113]:
#alias the CN to UID
import random
import uuid

uid = []
cn = []
for i in range(len(gdf)):
    rnd = random.Random()
    rnd.seed(gdf.iloc[i].CN)
    u_uid = uuid.UUID(int=rnd.getrandbits(128), version=4)
    uid.append(str(u_uid))
    cn.append(gdf.iloc[i].CN)
uid = np.array([uid]).T
cn = np.array([cn]).T

linked_df = pd.DataFrame(cn, columns = ['CN'])
linked_df['UID'] = uid

In [114]:
gdf = pd.merge(gdf, linked_df, on='CN')

### Iterate through FIA samples and collect image chips
Define both the area of interest and define the time range to filter images with. 

In [158]:
sentinel2_band_query = ['AOT','B01','B02','B03','B04','B05','B06','B07','B08','B09','B11','B12','B8A','SCL'] 

In [118]:
gdf_sample = gdf.sample(5)
sample_chips = collect_image_chips(gdf_sample)


In [131]:
item_names = [k for k,v in sample_chips[0][0].assets.items()]

In [156]:
#ones you want
band_query = item_names[:14]

In [157]:
band_query

['AOT',
 'B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B11',
 'B12',
 'B8A',
 'SCL']

In [ ]:
#this takes too long because of the size
#we need to use dask here

### Search the collection and choose an image to render

Use [pystac-client](https://github.com/stac-utils/pystac-client) to search for Sentinel 2 L2A data the range of the sample

In [137]:
i = gdf.iloc[0]
area_of_interest= i['AOI']
time_of_interest = i['DATERANGE']

In [138]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 100}},
)

# Check how many items were returned
items = list(search.get_items())
print(f"Returned {len(items)} Items")

Returned 112 Items


In [139]:
items[0].properties

{'datetime': '2018-12-29T16:37:01.024000Z',
 'platform': 'Sentinel-2A',
 'proj:epsg': 32616,
 'instruments': ['msi'],
 's2:mgrs_tile': '16RCU',
 'constellation': 'Sentinel 2',
 's2:granule_id': 'S2A_OPER_MSI_L2A_TL_ESRI_20201008T135827_A018384_T16RCU_N02.12',
 'eo:cloud_cover': 99.440094,
 's2:datatake_id': 'GS2A_20181229T163701_018384_N02.12',
 's2:product_uri': 'S2A_MSIL2A_20181229T163701_N0212_R083_T16RCU_20201008T135825.SAFE',
 's2:datastrip_id': 'S2A_OPER_MSI_L2A_DS_ESRI_20201008T135827_S20181229T163702_N02.12',
 's2:product_type': 'S2MSI2A',
 'sat:orbit_state': 'descending',
 's2:datatake_type': 'INS-NOBS',
 's2:generation_time': '2020-10-08T13:58:25.141Z',
 'sat:relative_orbit': 83,
 's2:water_percentage': 0.015338,
 's2:mean_solar_zenith': 56.0844236131173,
 's2:mean_solar_azimuth': 160.255674019091,
 's2:processing_baseline': '02.12',
 's2:snow_ice_percentage': 0.541919,
 's2:vegetation_percentage': 0.0,
 's2:thin_cirrus_percentage': 0.025899,
 's2:cloud_shadow_percentage': 0.

We can now work directly with the [PySTAC](https://github.com/stac-utils/pystac) Items returned by the API. Here we find the least cloudy of the bunch.

In [140]:
least_cloudy_item = sorted(items, key=lambda item: eo.ext(item).cloud_cover)[0]

print(
    f"Choosing {least_cloudy_item.id} from {least_cloudy_item.datetime.date()}"
    f" with {eo.ext(least_cloudy_item).cloud_cover}% cloud cover"
)

Choosing S2B_MSIL2A_20170930T164439_R083_T16RCU_20210202T121305 from 2017-09-30 with 0.130034% cloud cover


Get the URL to the 20m resolution visual [Cloud Optimized GeoTIFF](https://www.cogeo.org/) image.

In [142]:
asset_href = least_cloudy_item.assets["visual"].href
signed_href = pc.sign(asset_href)

### Render our AOI from this image

In [143]:
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image

In [144]:
with rasterio.open(signed_href) as ds:
    aoi_bounds = features.bounds(area_of_interest)
    warped_aoi_bounds = warp.transform_bounds("epsg:4326", ds.crs, *aoi_bounds)
    aoi_window = windows.from_bounds(transform=ds.transform, *warped_aoi_bounds)
    band_data = ds.read(window=aoi_window)

rasterio gives us data band-interleave format; transpose to pixel interleave, and downscale the image data for plotting.

In [145]:
band_data.shape

(3, 104, 104)